# Step II : Non-linear classification

In this example, the set of coordinates are now separated using a parabolic curve. This kind of problem cannot be solved with a single neuron and a deeper network will be needed.


Definition of a training data where the two classes are simply defined by Y<aX²+bX+c or Y>aX²+bX+c. The size of the 
training data set is 500. On top of it a Validation data set and a Testing data set are also defined, both of them
being of size 200.

In [ ]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt
    
def Training_set_parabole(a,b,c,N):
    
    n = 0
    Data = np.zeros([N+400,2])
    Labels = np.zeros([N+400,1])
      
    for n in range(0,N+400):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        Data[n,0] = x
        Data[n,1] = y
        
        if y < a*x*x+b*x+c :
            Labels[n,0] = 1
        else : 
            Labels[n,0] = 0

    Training_data = Data[:N,]
    Training_label = Labels[:N,]
    Validation_data = Data[N:N+200,]
    Validation_label = Labels[N:N+200,]
    Testing_data = Data[N+200:,]
    Testing_label = Labels[N+200:,]
        
    return Training_data, Testing_data, Validation_data, Training_label, Validation_label, Testing_label

def Training_set_clusters(N):
    
    n = 0
    Data = np.zeros([N+400,2])
    Labels = np.zeros([N+400,1])  

The training/validation/testing sets are defined below. The training set is then plot using two different colors
to distinguish the two classes


In [ ]:
a = 2;
b = 0;
c = -0.4;
N = 500;
    
Training_data, Testing_data, Validation_data, Training_label,Validation_label, Testing_label = Training_set_parabole(a,b,c,N) 
    
X0 = []
X1 = []
Y0 = []
Y1 = []
for n in range(0,N):
    x = Training_data[n,0]
    if Training_label[n]==0:
        X0.append(Training_data[n,0])
        Y0.append(Training_data[n,1])
    else:
        X1.append(Training_data[n,0])
        Y1.append(Training_data[n,1])

X = np.arange(-.8, .85, 0.1)
Y = a*X*X + b*X + c          

plt.plot(X,Y,'--k')
plt.plot(X0, Y0, 'r.')
plt.plot(X1, Y1, 'b.')
plt.axis('square')
plt.show() 

Define the architecture of the model. Here we will start with a single neuron and then build a more complex network. The input are however the same : the X and Y coordinates. Again, since we are working with two classes, the activation function is "sigmoid" and the loss function "binary cross-entropy". In the end the distinction between the two classes will be made on the base of whether the output will be below or above 0,5.

Note that the function "model.summary" will return a complete description of your network, highlighting the number of trainable parameters.


In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(1, activation='relu', input_shape=(2,)))

model.compile(optimizer = optimizers.sgd(lr=5e-2), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Training of the model. The number of Epoch and the minibatch size are defined below. The results at each iteraction
are saved in order to compare the accuracy calculated for the training set and for the validation set. These data are
saved in the variable history.


In [ ]:
history = model.fit(Training_data,
                    Training_label,
                    epochs = 20,
                    batch_size = 1,
                    validation_data = (Validation_data, Validation_label))

The accuracy of the model is tested using the testing set of data.

In [ ]:
Results = model.predict(Testing_data)
Length = len(Results)

X0 = []
X1 = []
Y0 = []
Y1 = []
for n in range(0,Length):

    if Results[n]<=0.5:
        X0.append(Testing_data[n,0])
        Y0.append(Testing_data[n,1])
    else:
        X1.append(Testing_data[n,0])
        Y1.append(Testing_data[n,1])

X = np.arange(-.8, .85, 0.1)
Y = a*X*X + b*X + c        
        
plt.plot(X0, Y0, 'r.')
plt.plot(X1, Y1, 'b.')
plt.plot(X,Y,'--k')
plt.axis('square')
plt.show() 

In the same way, using the model.evaluate function you can test the accuracy of the model when working on the testing
set. The second number returns the average accuracy.

In [ ]:
Predication_accuracy = model.evaluate(Testing_data, Testing_label)
print(Predication_accuracy)

Below the accuracy for the training and validation sets are plotted

In [ ]:
history_dict = history.history

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

n = len(acc_values)
epochs = range(1, n+1)

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Below the validation loss for the training and validation sets are plotted

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

A plot is obtained to illustrate how the data are modified by the network in order to classify them.

In [ ]:
Results = model.predict(Testing_data)

X0 = []
X1 = []
Y0 = []
Y1 = []
for n in range(0,Length):

    if Results[n]<=0.5:
        X0.append(Results[n])
        Y0.append(Testing_data[n,0])
    else:
        X1.append(Results[n])
        Y1.append(Testing_data[n,0])
        
plt.plot(Y0, X0, 'r.')
plt.plot(Y1, X1, 'b.')
plt.ylabel('Model output')
plt.xlabel('X position')
plt.axis('square')
plt.show()